### Objective

The objective is to predict the `total_minutes` a order takes to complete, where the rows not containing a `total_minutes` value should be set aside as a part of the submission file, containing the `order_id` with the predicted values. 

As we are interested in seeing how you attacked the problem, we also ask you to include your code together with the submission file. The code needs to be well documented, explaining the decisions made. With these explanations, we will be looking at everything from how the data was processed, features used to the completed model and predictions.

## Data

In this repository, we have included data representing the order, shopper and the stores branch. 

### File description and data fields
***order_products.csv:***
- order_id: ID of the order
- product_id: ID of the product
- quantity: The quantity ordered of this product
- buy_unit: The unit of the product (KG/UN)

***orders.csv:***
- order_id: ID of the order
- lat: The latitude of the delivery location
- lng: The longitude of the delivery location
- promised_time: The delivery time promised to the user
- on_demand: If true, the order was promised to be delivered in less than X minutes
- shopper_id: ID representing the shopper completed the order.
- stores_branch_id: ID of the stores branch
- total_minutes: The total minutes it took to complete the order (label)

***shopper.csv***
- shopper_id: ID of the shopper
- seniority: The experience level of the shopper.
- found_rate: Percentage of products found by shopper historical.
- picking_speed: Historical picking speed, products pr minutes.
- accepted_rate: Percentage of orders historically accepted by shopper
- rating: client rating of shopper

***storesbranch.csv:***
- stores_branch_id: ID of the stores branch
- stores: ID representing the stores
- lat: Latitude of the branch location
- lng: Longitude of the branch location

*All the data has been anonymized*

In [1]:
%matplotlib inline
import pandas as pd
pd.set_option("display.max_columns", 30)
import numpy as np
import os
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="iair_linker_cornershop_challenge")
from geopy.distance import geodesic
import time
from datetime import datetime

In [2]:
start_notebook = time.time()

In [3]:
def get_na(df):
    qsna=df.shape[0]-df.isnull().sum(axis=0)
    qna=df.isnull().sum(axis=0)
    ppna=round(100*(df.isnull().sum(axis=0)/df.shape[0]),2)
    aux= {'datos sin NAs en q': qsna, 'Na en q': qna ,'Na en %': ppna}
    na=pd.DataFrame(data=aux)
    return na.sort_values(by='Na en %',ascending=False)

def get_location(df):
    ind = []
    locations = []
    for index,row in df.iterrows():
        try:
            ind.append(index)
            location = geolocator.reverse([row.lat,row.lng])
            locations.append(location)
        except:
            time.sleep(120) # cuando falla por  esperando le digo que espere
            ind.append(index)
            location = geolocator.reverse([row.lat,row.lng])
            locations.append(location)
    return ind,locations

def get_distances(df):
    ind = []
    distances = []
    for index , row in df.iterrows():
        try:
            ind.append(index)
            distances.append(geodesic((row.orders_lat,row.orders_lng), (row.stores_lat,row.stores_lng)).km)
        except:
            time.sleep(30) # cuando falla por  esperando le digo que espere
            ind.append(index)
            distances.append(geodesic((row.orders_lat,row.orders_lng), (row.stores_lat,row.stores_lng)).km)
    return ind , distances

In [4]:
# set the working directory of my local 
# you need to change it if you are going to run it on your local machine, writing the folder where the raw data of the challenge is
os.chdir('/Users/iairlinker/Documents/repos/challenge - cornershop/data')

In [5]:
# Loading Order Products dataset
order_products = pd.read_csv('raw/order_products.csv', sep=',')
print(order_products.shape)
order_products.head()

(198500, 4)


,order_id,product_id,quantity,buy_unit
0,47099653730fb1b76537fc10ad876255,c1244453d731c77416cb4766e3bd76cb,1.0,UN
1,689d8866915acf87e851c2591a23a82f,43cc2b100bec640fe563cd16f2db669f,1.0,KG
2,f26d16bf6f38c9e31d0be877f4013a9e,b8f880759d014134e272d881d49989a2,1.0,UN
3,161ccc896835ab41761b0e726becb6b1,dbc062b9bef805d27a6f4bea7edfe1f1,1.0,UN
4,4713deca10bb5db98fae150b52d61fc0,93a060f269bb569398921100f84c519a,2.0,UN


In [6]:
get_na(order_products)

,datos sin NAs en q,Na en q,Na en %
order_id,198500,0,0.0
product_id,198500,0,0.0
quantity,198500,0,0.0
buy_unit,198500,0,0.0


In [7]:
order_products['quantity_unit'] = order_products.quantity.astype(str) + order_products.buy_unit

In [8]:
order_products.buy_unit.unique()

array(['UN', 'KG'], dtype=object)

In [9]:
order_products.to_csv('interim/order_products.csv',sep=';',index=False)
order_products.head(3)

,order_id,product_id,quantity,buy_unit,quantity_unit
0,47099653730fb1b76537fc10ad876255,c1244453d731c77416cb4766e3bd76cb,1.0,UN,1.0UN
1,689d8866915acf87e851c2591a23a82f,43cc2b100bec640fe563cd16f2db669f,1.0,KG,1.0KG
2,f26d16bf6f38c9e31d0be877f4013a9e,b8f880759d014134e272d881d49989a2,1.0,UN,1.0UN


In [10]:
orders = pd.read_csv('raw/orders.csv', sep=',')
print(orders.shape)
print(orders.shape)
orders.head()

(10000, 8)
(10000, 8)


,order_id,lat,lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,2019-10-19 20:00:00+00:00,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822


In [11]:
get_na(orders)

,datos sin NAs en q,Na en q,Na en %
total_minutes,8000,2000,20.0
order_id,10000,0,0.0
lat,10000,0,0.0
lng,10000,0,0.0
promised_time,10000,0,0.0
on_demand,10000,0,0.0
shopper_id,10000,0,0.0
store_branch_id,10000,0,0.0


In [12]:
#model_data['promised_time_minutes'] = model_data.promised_time.apply(lambda x: x.split('+')[0][0:-3])
orders['promised_time_hours'] = orders.promised_time.apply(lambda x: (datetime.strptime(str(x).split('+')[0][0:-6], '%Y-%m-%d %H')))
orders['promised_time_id'] = orders.promised_time_hours.apply(lambda x: str(x.day) + '-' + str(x.hour))
bins = [0, 60, 80, 100, np.inf]
names = ['menor a 60','entre 60 y 80', 'entre 80 y 100', 'mayor a 100']
orders['total_minutes_range'] = pd.cut(orders['total_minutes'], bins, labels=names)
orders.head(2)

,order_id,lat,lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes,promised_time_hours,promised_time_id,total_minutes_range
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,2019-10-18 20:00:00,18-20,entre 60 y 80
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,2019-10-19 01:00:00,19-1,menor a 60


In [13]:
start = time.time()

ind, locations = get_location(orders)

end = time.time()
total_time = end - start
print(total_time)

5062.644371986389


In [14]:
orders.rename(columns=({'lat':'orders_lat', 'lng':'orders_lng'}), inplace=True)
orders['orders_location'] = locations
# agrego una variable de string que explica el True o False de la variable on_demand para graficar
orders['sla'] = orders.on_demand.apply(lambda x : "promised in less than X min" if x is True else "not promised")
# Al analizar la estructura que me pasa como resultado geocipy puedo detectar que hay dos casos:
# el más común es que hay 4 elementos de atrás hacia adelante para llegar a la comuna
# el menos común es que solo hay 3 porque no está el postalcode y viene así: ['Colina', ' Provincia de Chacabuco', ' Región Metropolitana de Santiago', ' Chile']
# hago el algoritmo para sacar la comuna
aux = orders.orders_location.apply(lambda x: x[0].split(','))
comunas = []
for arr in aux:
    try:
        int(arr[-2].strip())
        comunas.append((arr[-5].strip()))
    except:
        comunas.append((arr[-4].strip()))
orders['order_district'] = comunas
del aux, comunas
orders.to_csv('interim/orders.csv',sep=';',index= False)
orders.head(3)


,order_id,orders_lat,orders_lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes,promised_time_hours,promised_time_id,total_minutes_range,orders_location,sla,order_district
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,2019-10-18 20:00:00,18-20,entre 60 y 80,"(5098, Avenida Las Torres, Conjunto San Luis, ...",promised in less than X min,Peñalolén
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,2019-10-19 01:00:00,19-1,menor a 60,"(1497, Pasaje Mateo De Toro y Zambrano, La Rei...",not promised,La Reina
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,2019-10-19 14:00:00,19-14,NaN,"(Altair, Población Naval Las Salinas, Viña del...",promised in less than X min,Viña del Mar


In [15]:
shoppers = pd.read_csv('raw/shoppers.csv', sep=',')
print(shoppers.shape)
shoppers.head(3)

(2864, 6)


,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
0,1fc20b0bdf697ac13dd6a15cbd2fe60a,41dc7c9e385c4d2b6c1f7836973951bf,0.8606,1.94,1.00,4.87
1,e1c679ac73a69c01981fdd3c5ab8beda,6c90661e6d2c7579f5ce337c3391dbb9,0.8446,1.23,0.92,4.92
2,09d369c66ca86ebeffacb133410c5ee1,6c90661e6d2c7579f5ce337c3391dbb9,0.8559,1.56,1.00,4.88


In [16]:
get_na(shoppers)

,datos sin NAs en q,Na en q,Na en %
found_rate,2763,101,3.53
rating,2780,84,2.93
accepted_rate,2837,27,0.94
shopper_id,2864,0,0.00
seniority,2864,0,0.00
picking_speed,2864,0,0.00


In [17]:
shoppers.seniority.unique()

array(['41dc7c9e385c4d2b6c1f7836973951bf',
       '6c90661e6d2c7579f5ce337c3391dbb9',
       '50e13ee63f086c2fe84229348bc91b5b',
       'bb29b8d0d196b5db5a5350e5e3ae2b1f'], dtype=object)

In [18]:
# Renaming seniority column values 
shopper_seniority = {'6c90661e6d2c7579f5ce337c3391dbb9': 'ADVANCED',
                     '50e13ee63f086c2fe84229348bc91b5b': 'INTERMEDIATE',
                     '41dc7c9e385c4d2b6c1f7836973951bf': 'BEGINNER',
                     'bb29b8d0d196b5db5a5350e5e3ae2b1f': 'REVIEW'}

shoppers['seniority'] = shoppers['seniority'].map(shopper_seniority)
shoppers.seniority.unique()

array(['BEGINNER', 'ADVANCED', 'INTERMEDIATE', 'REVIEW'], dtype=object)

In [19]:
shoppers.to_csv('interim/shoppers.csv',sep=';', index= False)

In [20]:
stores = pd.read_csv('raw/storebranch.csv', sep=',')
print(stores.shape)
stores.head(3)

(476, 4)


,store_branch_id,store_id,lat,lng
0,aff1621254f7c1be92f64550478c56e6,92cc227532d17e56e07902b254dfad10,-33.422497,-70.609231
1,56352739f59643540a3a6e16985f62c7,0336dcbab05b9d5ad24f4333c7658a0e,-33.385484,-70.555579
2,7d04bbbe5494ae9d2f5a76aa1c00fa2f,9bf31c7ff062936a96d3c8bd1f8f2ff3,-33.416579,-70.565224


In [21]:
get_na(stores)

,datos sin NAs en q,Na en q,Na en %
store_branch_id,476,0,0.0
store_id,476,0,0.0
lat,476,0,0.0
lng,476,0,0.0


In [22]:
start = time.time()

ind,locations = get_location(stores)
stores['stores_location'] = locations

end = time.time()
total_time = end - start
print(total_time)

242.42800307273865


In [23]:
start = time.time()

aux = stores.stores_location.apply(lambda x: x[0].split(','))
comunas = []
for arr in aux:
    try:
        int(arr[-2].strip())
        comunas.append((arr[-5].strip()))
    except:
        comunas.append((arr[-4].strip()))
stores['store_district'] = comunas
del aux, comunas

end = time.time()
total_time = end - start
print(total_time)

0.005023002624511719


In [24]:
stores.rename(columns=({'lat':'stores_lat', 'lng':'stores_lng'}), inplace=True)
stores.to_csv('interim/stores.csv',sep=';', index=False)
print(stores.shape)
stores.head(2)

(476, 6)


,store_branch_id,store_id,stores_lat,stores_lng,stores_location,store_district
0,aff1621254f7c1be92f64550478c56e6,92cc227532d17e56e07902b254dfad10,-33.422497,-70.609231,"(2206, Avenida Nueva Providencia, Sanhattan, P...",Providencia
1,56352739f59643540a3a6e16985f62c7,0336dcbab05b9d5ad24f4333c7658a0e,-33.385484,-70.555579,"(Puerto Callao, Avenida Vitacura, El Dorado, V...",Vitacura


In [25]:
end_notebook = time.time()
total_time = end_notebook - start_notebook
print(total_time)

5307.715092897415


## Creando el tablon de modelamiento

In [26]:
start_notebook = time.time()

In [27]:
# Loading Order Products dataset
#orders = pd.read_csv('interim/orders.csv', sep=';', index_col=False)
#order_products = pd.read_csv('interim/order_products.csv', sep=';')
product_quantity_by_order = order_products.groupby(["order_id",'buy_unit'])["product_id"].nunique().reset_index()
product_quantity_by_order.rename(columns=({"product_id":'quantity'}), inplace=True)
aux = pd.pivot_table(product_quantity_by_order,index=["order_id"], columns=["buy_unit"]).fillna(0)
aux.columns = ['KG','UN']
# Merging orders with shoppers on "shopper_id"
df = pd.merge(orders.set_index('order_id'), aux, how='left',left_index=True, right_index=True)
df.reset_index(inplace=True)
del product_quantity_by_order, aux
print(df.shape)
get_na(df)

(10000, 16)


,datos sin NAs en q,Na en q,Na en %
total_minutes,8000,2000,20.00
total_minutes_range,8000,2000,20.00
KG,9978,22,0.22
UN,9978,22,0.22
order_id,10000,0,0.00
orders_lat,10000,0,0.00
orders_lng,10000,0,0.00
promised_time,10000,0,0.00
on_demand,10000,0,0.00
shopper_id,10000,0,0.00


In [28]:
#stores = pd.read_csv('interim/stores.csv', sep=';')
# Merging orders with stores on "stores_branch_id"
df = pd.merge(df, stores, how='left', on='store_branch_id')
print(df.shape)
get_na(df)

(10000, 21)


,datos sin NAs en q,Na en q,Na en %
total_minutes_range,8000,2000,20.00
total_minutes,8000,2000,20.00
UN,9978,22,0.22
KG,9978,22,0.22
orders_location,10000,0,0.00
stores_location,10000,0,0.00
stores_lng,10000,0,0.00
stores_lat,10000,0,0.00
store_id,10000,0,0.00
order_district,10000,0,0.00


In [29]:
#shoppers = pd.read_csv('interim/shoppers.csv', sep=';')
# Merging orders with stores on "stores_branch_id"
df = pd.merge(df, shoppers, how='left', on='shopper_id')
print(df.shape)
get_na(df)

(10000, 26)


,datos sin NAs en q,Na en q,Na en %
total_minutes,8000,2000,20.00
total_minutes_range,8000,2000,20.00
found_rate,9800,200,2.00
rating,9837,163,1.63
accepted_rate,9954,46,0.46
KG,9978,22,0.22
UN,9978,22,0.22
picking_speed,10000,0,0.00
seniority,10000,0,0.00
store_district,10000,0,0.00


In [30]:
start = time.time()

ind,order_stores_distance = get_distances(df)
df['order_stores_distance'] = order_stores_distance

end = time.time()
total_time = end - start
print(total_time)

3.7016022205352783


In [31]:
## separo los datos en aquellos que tengo registro de la variable de tiempo que demoró el viaje y las que no
results  = df.loc[pd.isnull(df.total_minutes)]
model_data = df.loc[pd.notnull(df.total_minutes)]
print(model_data.shape)
model_data.head(2)

(8000, 27)


,order_id,orders_lat,orders_lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes,promised_time_hours,promised_time_id,total_minutes_range,orders_location,sla,order_district,KG,UN,store_id,stores_lat,stores_lng,stores_location,store_district,seniority,found_rate,picking_speed,accepted_rate,rating,order_stores_distance
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,2019-10-18 20:00:00,18-20,entre 60 y 80,"(5098, Avenida Las Torres, Conjunto San Luis, ...",promised in less than X min,Peñalolén,4.0,15.0,c4ca4238a0b923820dcc509a6f75849b,-33.485280,-70.57925,"(Mall Paseo Quilín, Peñalolén, Provincia de Sa...",Peñalolén,ADVANCED,0.9024,1.30,0.92,4.76,1.818407
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,2019-10-19 01:00:00,19-1,menor a 60,"(1497, Pasaje Mateo De Toro y Zambrano, La Rei...",not promised,La Reina,0.0,5.0,c4ca4238a0b923820dcc509a6f75849b,-33.441246,-70.53545,"(Lider, 9140, Avenida Príncipe de Gales, La Re...",La Reina,BEGINNER,0.7610,2.54,0.92,4.96,1.938544


In [32]:
get_na(results)

,datos sin NAs en q,Na en q,Na en %
total_minutes,0,2000,100.00
total_minutes_range,0,2000,100.00
found_rate,1969,31,1.55
rating,1975,25,1.25
UN,1995,5,0.25
accepted_rate,1995,5,0.25
KG,1995,5,0.25
order_id,2000,0,0.00
picking_speed,2000,0,0.00
seniority,2000,0,0.00


In [33]:
# cantidad de datos que me quedan luego de eliminar todos los NA
print(results.dropna(subset=['rating', 'found_rate', 'accepted_rate', 'KG', 'UN']).shape[0])
100*((results.dropna(subset=['rating', 'found_rate', 'accepted_rate', 'KG', 'UN']).shape[0]/results.shape[0]))

1960


98.0

In [34]:
model_data.to_csv('interim/model_data.csv',sep = ';', index= False)
print(model_data.shape)
model_data.head(3)

(8000, 27)


,order_id,orders_lat,orders_lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes,promised_time_hours,promised_time_id,total_minutes_range,orders_location,sla,order_district,KG,UN,store_id,stores_lat,stores_lng,stores_location,store_district,seniority,found_rate,picking_speed,accepted_rate,rating,order_stores_distance
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,2019-10-18 20:00:00,18-20,entre 60 y 80,"(5098, Avenida Las Torres, Conjunto San Luis, ...",promised in less than X min,Peñalolén,4.0,15.0,c4ca4238a0b923820dcc509a6f75849b,-33.485280,-70.579250,"(Mall Paseo Quilín, Peñalolén, Provincia de Sa...",Peñalolén,ADVANCED,0.9024,1.30,0.92,4.76,1.818407
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,2019-10-19 01:00:00,19-1,menor a 60,"(1497, Pasaje Mateo De Toro y Zambrano, La Rei...",not promised,La Reina,0.0,5.0,c4ca4238a0b923820dcc509a6f75849b,-33.441246,-70.535450,"(Lider, 9140, Avenida Príncipe de Gales, La Re...",La Reina,BEGINNER,0.7610,2.54,0.92,4.96,1.938544
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742,2019-10-18 21:00:00,18-21,menor a 60,"(Basel, Alpes Suizos, Lo Barnechea, Provincia ...",promised in less than X min,Lo Barnechea,0.0,1.0,f718499c1c8cef6730f9fd03c8125cab,-33.355258,-70.537787,"(10660, Avenida José Alcalde Délano, Las Patag...",Lo Barnechea,BEGINNER,0.8776,2.80,0.96,4.76,3.815982


In [35]:
results.to_csv('interim/results.csv',sep = ';',index= False)
print(results.shape)
results.head(3)

(2000, 27)


,order_id,orders_lat,orders_lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes,promised_time_hours,promised_time_id,total_minutes_range,orders_location,sla,order_district,KG,UN,store_id,stores_lat,stores_lng,stores_location,store_district,seniority,found_rate,picking_speed,accepted_rate,rating,order_stores_distance
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,2019-10-19 14:00:00,19-14,NaN,"(Altair, Población Naval Las Salinas, Viña del...",promised in less than X min,Viña del Mar,0.0,5.0,c4ca4238a0b923820dcc509a6f75849b,-33.008213,-71.545615,"(Mall Espacio Urbano, 961, Avenida Benidorm, P...",Viña del Mar,INTERMEDIATE,0.8313,2.57,0.76,4.92,2.351240
5,9bf29b56619fcaf60b52690a848e10bb,-33.330724,-70.547074,2019-10-18 23:47:00+00:00,True,61e4ad15c3ff928840ebd34407055b33,33e75ff09dd601bbe69f351039152189,NaN,2019-10-18 23:00:00,18-23,NaN,"(Condominio Los Monjes, Lo Barnechea, Provinci...",promised in less than X min,Lo Barnechea,0.0,10.0,a87ff679a2f3e71d9181a67b7542122c,-33.357558,-70.515415,"(Mall Portal La Dehesa, 1445, Avenida La Dehes...",Lo Barnechea,ADVANCED,0.8946,1.82,1.00,4.84,4.188461
9,299d948a5fd2cf2a921894b9bd24b94e,-33.349922,-70.522841,2019-10-18 21:54:00+00:00,True,1456fc09701783b29f69e8f68c029879,1679091c5a880faf6fb5e6087eb1b2dc,NaN,2019-10-18 21:00:00,18-21,NaN,"(2376, Camino Central, La Dehesa, Lo Barnechea...",promised in less than X min,Lo Barnechea,4.0,44.0,c4ca4238a0b923820dcc509a6f75849b,-33.370765,-70.512420,"(Lider, 12916, Avenida Las Condes, Lo Barneche...",Lo Barnechea,ADVANCED,0.9363,1.67,1.00,4.84,2.506907


In [36]:
end_notebook = time.time()
total_time = end_notebook - start_notebook
print(total_time)

4.837050914764404
